# Git internals study session, 22 February 2023

## 0. The `.git/` subdir 

In [1]:
%pwd

'/Users/eric.fulmer/code/git-internals-study-session/nbs'

In [2]:
%cd ..

/Users/eric.fulmer/code/git-internals-study-session


In [3]:
import hashlib
import os
import pathlib
import zlib

In [4]:
GIT_DIRECTORY = pathlib.PosixPath(".git/")

OBJECTS_ROOT_DIR = GIT_DIRECTORY / "objects"

OLD_FILE_ID = "2bfc82f7e00838e0e39dde7b77c4df3b4817f091"

print(GIT_DIRECTORY.exists())
print("\n".join([str(x) for x in GIT_DIRECTORY.iterdir()]))

True
.git/config
.git/objects
.git/HEAD
.git/info
.git/logs
.git/description
.git/hooks
.git/refs
.git/index
.git/COMMIT_EDITMSG


In [5]:
!ls -ahl .git/

total 40
drwxr-xr-x  12 eric.fulmer  1052847317   384B Feb 21 16:45 .
drwxr-xr-x   9 eric.fulmer  1052847317   288B Feb 20 16:16 ..
-rw-r--r--   1 eric.fulmer  1052847317    24B Feb 21 16:44 COMMIT_EDITMSG
-rw-r--r--   1 eric.fulmer  1052847317    21B Feb 21 16:45 HEAD
-rw-r--r--   1 eric.fulmer  1052847317   320B Feb 21 16:45 config
-rw-r--r--   1 eric.fulmer  1052847317    73B Feb 20 09:36 description
drwxr-xr-x  15 eric.fulmer  1052847317   480B Feb 20 09:36 hooks
-rw-r--r--   1 eric.fulmer  1052847317   539B Feb 21 16:44 index
drwxr-xr-x   3 eric.fulmer  1052847317    96B Feb 20 09:36 info
drwxr-xr-x   4 eric.fulmer  1052847317   128B Feb 20 09:36 logs
drwxr-xr-x  24 eric.fulmer  1052847317   768B Feb 21 16:44 objects
drwxr-xr-x   5 eric.fulmer  1052847317   160B Feb 21 16:45 refs


In [6]:
with open(GIT_DIRECTORY / "HEAD", "r") as f:
    print(f.read())

ref: refs/heads/main



## 1. Git Objects I - Blobs

In [7]:
[
    f".git/objects/{d.name}/{f.name}" for d in (GIT_DIRECTORY / "objects").iterdir() if d.is_dir()
    for f in d.iterdir() if f.is_file()
]

['.git/objects/68/bc17f9ff2104a9d7b6777058bb4c343ca72609',
 '.git/objects/34/292f14b07a4603664a6ff4a0285a22d214754a',
 '.git/objects/9d/1dcfdaf1a6857c5f83dc27019c7600e1ffaff8',
 '.git/objects/a4/d2d0e56b9eb8300532f9d9e1bf1e38dce76011',
 '.git/objects/b5/a99e2b88b7fdb8fb3e54a1df2ce801acecfeb5',
 '.git/objects/d8/e0be6dc085079b872c14999a1783f47ca1311f',
 '.git/objects/ab/1964713b60fdfbe235a362b3ec0f2ef9c066c0',
 '.git/objects/c8/643ec5d032481de3f2fcc4abf03244cf2acc75',
 '.git/objects/c6/266c7a5b2b5825ff627216e8133135ff0ec496',
 '.git/objects/43/e6a9df6f7da386ebab015f923823325b7cb108',
 '.git/objects/06/fec98b0b438c682f5d9a53f96e1ce87dcf0ba8',
 '.git/objects/64/ba6ed86228c76b5090cef85c65d91a770c7a99',
 '.git/objects/bf/344ad78d35d1ac06d539e6720b02a15800a385',
 '.git/objects/e6/9de29bb2d1d6434b8b29ae775ad8c2e48c5391',
 '.git/objects/f6/145c9b7ecff3b9eccdb17cabc0b2cd1b2e2c10',
 '.git/objects/e7/d42dd6ea9c7d33673f6f81c171807137356eb5',
 '.git/objects/46/c51474515d3bc9c9388040206e58db2f2ff080

In [ ]:
!ls -alt .git/objects

In [ ]:
ls -alt .git/objects/d8

In [8]:
hasher = hashlib.sha1()

with open("pyproject.toml", "r") as f:
    contents = f.read()
    header = f"blob {len(contents)}\0"
    git_object = header + contents

hasher.update(bytes(git_object.encode('utf-8')))
print(hasher.hexdigest())
head_pyproject_id = hasher.hexdigest()

b5a99e2b88b7fdb8fb3e54a1df2ce801acecfeb5


In [9]:
!ls .git/objects/b5

a99e2b88b7fdb8fb3e54a1df2ce801acecfeb5


In [10]:
with open(OBJECTS_ROOT_DIR / head_pyproject_id[:2] / head_pyproject_id[2:], "rb") as file_contents:
    git_pyproject = file_contents.read()
    print(git_pyproject)

b"x\x01M\x90OK\x031\x14\xc4=\xe7S<r\x95\r\xdd\xaa\x85\x82\x8a\x82[O\x82\xe8\xb1\xac\x90M^m4\xff|I\x84|{\xd3n)\xde\xc2\xe41\xf3\x9b\x99l\x98\xe0\xba_]ls\x08V\xc4\x80\x99\xea\xc8\xbct\x08w\xc0?M\xee\x8c\xcfH^\xda\xd4\xa5\\t\xed\x12\xa6d\x82\xe7\xec\x17\xe9\xf08\xdc\xf5b!\x16\x9ciL\x8aL\xcc'\xf5\xddIkA\xa3\x0b\x10v\xf0l2\x9c\xbd`\x17\x08\xa4?\x0b\xb0\x91\xb4\xc3\xac\xf6p\x0c\x81s\x88,y\x1f(\xb5\x90-\x1f\xc8(\xd8\x14\xeb\x90\xe0v\xb9\xbeY\xae\xd7\xab\xcba\x16\x1eJj8\xc2\x07\xc2h\xabh\xe0\xfb2\t\x15\xdc=\x1f\x995\n}:6z\xa5\x10\xc9`\x96T9#\x94z.\xfa6<>\xbd\x0c\xc2i\xce\xd8\xff)\x84\xc6\x88^\xa3W\x06\xd3\xc8bm4\xc7\xc6\x1fW\xa2\xef9\xfb*Mj8m\x83\x8f\xd3\x08\xcd`*\xc6\xea.\xd5\x94\xd1\x8d-\xe5\xa7\x18\xc2\xb9\xc3\xbcp\xa7\x1ag\x03\x9b\x0f'\xa9\xbe[\xc8\xc1d\xfen\xdc\x84\xc2\xc9\x14<U!\xa3\xe1\xec\x0f\x05\\\x8fM"


In [11]:
print(zlib.decompress(git_pyproject).decode("utf-8"))

blob 416 [tool.poetry]
name = "git-internals-study-session"
version = "1.0.0"
description = "Small demo of Git internals for an internal Farfetch study session"
authors = ["Eric Fulmer <2952996+EFulmer@users.noreply.github.com>"]
license = "Proprietary"
readme = "README.md"

[tool.poetry.dependencies]
python = "^3.11"
jupyter = "^1.0.0"


[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"



In [12]:
!cat pyproject.toml

[tool.poetry]
name = "git-internals-study-session"
version = "1.0.0"
description = "Small demo of Git internals for an internal Farfetch study session"
authors = ["Eric Fulmer <2952996+EFulmer@users.noreply.github.com>"]
license = "Proprietary"
readme = "README.md"

[tool.poetry.dependencies]
python = "^3.11"
jupyter = "^1.0.0"


[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"


### Sanity check

In [13]:
!git hash-object -t blob pyproject.toml

b5a99e2b88b7fdb8fb3e54a1df2ce801acecfeb5


In [14]:
!git cat-file -p b5a99e2b88b7fdb8fb3e54a1df2ce801acecfeb5

[tool.poetry]
name = "git-internals-study-session"
version = "1.0.0"
description = "Small demo of Git internals for an internal Farfetch study session"
authors = ["Eric Fulmer <2952996+EFulmer@users.noreply.github.com>"]
license = "Proprietary"
readme = "README.md"

[tool.poetry.dependencies]
python = "^3.11"
jupyter = "^1.0.0"


[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"


### Immutable object database!

In [15]:
!git cat-file -p $OLD_FILE_ID

[tool.poetry]
name = "git-internals-study-session"
version = "1.0.0"
description = "Small demo of Git internals for an internal Farfetch study session"
authors = ["Eric Fulmer <2952996+EFulmer@users.noreply.github.com>"]
license = "Proprietary"
readme = "README.md"
packages = [{include = "git_internals_study_session"}]

[tool.poetry.dependencies]
python = "^3.11"
jupyter = "^1.0.0"


[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"


## 1. Git Objects II - Tree

In [16]:
!git cat-file -p main^{tree}

100644 blob 68bc17f9ff2104a9d7b6777058bb4c343ca72609	.gitignore
040000 tree 06fec98b0b438c682f5d9a53f96e1ce87dcf0ba8	nbs
100644 blob ab1964713b60fdfbe235a362b3ec0f2ef9c066c0	poetry.lock
100644 blob b5a99e2b88b7fdb8fb3e54a1df2ce801acecfeb5	pyproject.toml


In [17]:
!ls -halt

total 232
drwxr-xr-x   5 eric.fulmer  1052847317   160B Feb 22 10:26 nbs
drwxr-xr-x  12 eric.fulmer  1052847317   384B Feb 21 16:45 .git
drwxr-xr-x   2 eric.fulmer  1052847317    64B Feb 20 16:16 .ipynb_checkpoints
drwxr-xr-x   9 eric.fulmer  1052847317   288B Feb 20 16:16 .
-rw-r--r--   1 eric.fulmer  1052847317   416B Feb 20 16:04 pyproject.toml
-rw-r--r--   1 eric.fulmer  1052847317   3.0K Feb 20 09:37 .gitignore
-rw-r--r--   1 eric.fulmer  1052847317   102K Feb 20 09:34 poetry.lock
-rw-r--r--   1 eric.fulmer  1052847317     7B Feb 20 09:33 .python-version
drwxr-xr-x  29 eric.fulmer  1052847317   928B Feb 20 09:31 ..


In [18]:
!git cat-file -p 68bc17f9ff2104a9d7b6777058bb4c343ca72609

# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class

# C extensions
*.so

# Distribution / packaging
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
share/python-wheels/
*.egg-info/
.installed.cfg
*.egg
MANIFEST

# PyInstaller
#  Usually these files are written by a python script from a template
#  before PyInstaller builds the exe, so as to inject date/other infos into it.
*.manifest
*.spec

# Installer logs
pip-log.txt
pip-delete-this-directory.txt

# Unit test / coverage reports
htmlcov/
.tox/
.nox/
.coverage
.coverage.*
.cache
nosetests.xml
coverage.xml
*.cover
*.py,cover
.hypothesis/
.pytest_cache/
cover/

# Translations
*.mo
*.pot

# Django stuff:
*.log
local_settings.py
db.sqlite3
db.sqlite3-journal

# Flask stuff:
instance/
.webassets-cache

# Scrapy stuff:
.scrapy

# Sphinx documentation
docs/_build/

# PyBuilder
.pybuilder/
target/

# Jupyter Notebook
.ipynb_checkpoints

# IPython
profile_default/
ipython

In [19]:
!git cat-file -p bf344ad78d35d1ac06d539e6720b02a15800a385

100644 blob e7d42dd6ea9c7d33673f6f81c171807137356eb5	Git Internals.ipynb
040000 tree 9d1dcfdaf1a6857c5f83dc27019c7600e1ffaff8	utils


## 2. Git Objects III - Commit

In [20]:
!git log --oneline

a4d2d0e (HEAD -> main, origin/main) last commit before pres
34292f1 add subdir for tree demo
d8e0be6 remove packages line from pyproject.toml
46c5147 add .gitignore
41dd0bd (tag: start) first commit


In [21]:
!git cat-file -p 34292f1

tree 43e6a9df6f7da386ebab015f923823325b7cb108
parent d8e0be6dc085079b872c14999a1783f47ca1311f
author Eric Fulmer <2952996+EFulmer@users.noreply.github.com> 1676927848 -0500
committer Eric Fulmer <2952996+EFulmer@users.noreply.github.com> 1676927848 -0500

add subdir for tree demo


In [22]:
!git cat-file -p 41dd0bd

tree 64ba6ed86228c76b5090cef85c65d91a770c7a99
author Eric Fulmer <2952996+EFulmer@users.noreply.github.com> 1676903814 -0500
committer Eric Fulmer <2952996+EFulmer@users.noreply.github.com> 1676903814 -0500

first commit


In [23]:
!git cat-file -p 64ba6ed86228c76b5090cef85c65d91a770c7a99

100644 blob ab1964713b60fdfbe235a362b3ec0f2ef9c066c0	poetry.lock
100644 blob 2bfc82f7e00838e0e39dde7b77c4df3b4817f091	pyproject.toml


## 2. Git Objects IV - Refs

In [24]:
%pwd

'/Users/eric.fulmer/code/git-internals-study-session'

In [25]:
!ls -alt .git/refs/

total 0
drwxr-xr-x   3 eric.fulmer  1052847317   96 Feb 21 16:45 remotes
drwxr-xr-x   5 eric.fulmer  1052847317  160 Feb 21 16:45 .
drwxr-xr-x  12 eric.fulmer  1052847317  384 Feb 21 16:45 ..
drwxr-xr-x   3 eric.fulmer  1052847317   96 Feb 21 16:45 heads
drwxr-xr-x   3 eric.fulmer  1052847317   96 Feb 21 10:47 tags


In [26]:
!ls -alt .git/refs/tags

total 8
drwxr-xr-x  5 eric.fulmer  1052847317  160 Feb 21 16:45 ..
drwxr-xr-x  3 eric.fulmer  1052847317   96 Feb 21 10:47 .
-rw-r--r--  1 eric.fulmer  1052847317   41 Feb 21 10:47 start


In [27]:
!cat .git/refs/tags/start

41dd0bdb604c7732ef6f9eb4b887a9bd5088b7df


In [28]:
!git cat-file -p 41dd0bdb604c7732ef6f9eb4b887a9bd5088b7df

tree 64ba6ed86228c76b5090cef85c65d91a770c7a99
author Eric Fulmer <2952996+EFulmer@users.noreply.github.com> 1676903814 -0500
committer Eric Fulmer <2952996+EFulmer@users.noreply.github.com> 1676903814 -0500

first commit


In [29]:
!ls -alt .git/refs/heads

total 8
drwxr-xr-x  5 eric.fulmer  1052847317  160 Feb 21 16:45 ..
drwxr-xr-x  3 eric.fulmer  1052847317   96 Feb 21 16:45 .
-rw-r--r--  1 eric.fulmer  1052847317   41 Feb 21 16:45 main


In [30]:
!cat .git/refs/heads/main

a4d2d0e56b9eb8300532f9d9e1bf1e38dce76011


## 3. Index

In [31]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   nbs/Git Internals.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.python-version

no changes added to commit (use "git add" and/or "git commit -a")


In [32]:
with open(".git/index", "rb") as f:
    index = f.read()

In [33]:
index

b'DIRC\x00\x00\x00\x02\x00\x00\x00\x05c\xf3\x85\xaf\x1ap\xc6\xc5c\xf3\x85\xaf\x1ap\xc6\xc5\x01\x00\x00\t\x01\x9bB_\x00\x00\x81\xa4/\xc5\x9b\x90>\xc1,\xd5\x00\x00\x0c\x06h\xbc\x17\xf9\xff!\x04\xa9\xd7\xb6wpX\xbbL4<\xa7&\t\x00\n.gitignore\x00\x00\x00\x00\x00\x00\x00\x00c\xf5;"\x1bF\x7fPc\xf5;"\x1bF\x7fP\x01\x00\x00\t\x01\x9bC\xa3\x00\x00\x81\xa4/\xc5\x9b\x90>\xc1,\xd5\x00\x00j\xf6\xc8d>\xc5\xd02H\x1d\xe3\xf2\xfc\xc4\xab\xf02D\xcf*\xccu\x00\x17nbs/Git Internals.ipynb\x00\x00\x00c\xf3\xe3Q,u\xda\xccc\xf3\xe3Q,u\xda\xcc\x01\x00\x00\t\x01\x9b\x8c\xbf\x00\x00\x81\xa4/\xc5\x9b\x90>\xc1,\xd5\x00\x00\x00\x00\xe6\x9d\xe2\x9b\xb2\xd1\xd6CK\x8b)\xaewZ\xd8\xc2\xe4\x8cS\x91\x00\x15nbs/utils/__init__.py\x00\x00\x00\x00\x00c\xf3\x85\x04.\xdc\x84$c\xf3\x85\x04.\xdc\x84$\x01\x00\x00\t\x01\x9b\x0bv\x00\x00\x81\xa4/\xc5\x9b\x90>\xc1,\xd5\x00\x01\x99x\xab\x19dq;`\xfd\xfb\xe25\xa3b\xb3\xec\x0f.\xf9\xc0f\xc0\x00\x0bpoetry.lock\x00\x00\x00\x00\x00\x00\x00c\xf3\xe0h\x16\xc8\x03\x87c\xf3\xe0h\x16\xc4\xc0\x1f\x01

In [34]:
print(zlib.decompress(index).decode("utf-8"))

error: Error -3 while decompressing data: incorrect header check

Documentation on the format of the index: https://git-scm.com/docs/index-format

# Fin
Read more:
* Pro Git chapter about Git internals: https://git-scm.com/book/en/v2/Git-Internals-Plumbing-and-Porcelain
* libgit2 (Python library for interacting with Git): https://github.com/libgit2/pygit2
* Online documentation reference: https://git-scm.com/docs/
* Git Index Format: https://git-scm.com/docs/index-format